## Example for Creating Accuracy Assessment Points - Map Visualization

In [1]:
# Import GEE & initialize
import os
import ee
import geemap
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# import seaborn as sns
# sns.set_theme(style="darkgrid")

# Initialize GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Add / Import Data sets

In [2]:
# Data sets
classified = ee.Image("projects/ee-abnatcap/assets/sargassum/classclipped_20190402")
nearshore_mask = ee.FeatureCollection("projects/ee-abnatcap/assets/sargassum/S2_sargassum_mask")
sargassum = classified.select("sargassum").eq(1).selfMask();
print(classified.getInfo())

S2bands = ee.List(['B2', 'B3', 'B4', 'B5', 'B8', 'B8A', 'B11', 'B12', 'QA60'])
QR_multipoint = ee.Geometry.MultiPoint(
    [[-87.4, 21.2], [-86.6, 21.2], [-87.4, 20.3], [-86.6, 20.3], [-87.4, 19.3], [-87.4, 18.3]])
s2mosaic = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(QR_multipoint) \
        .filterDate('2019-04-02', '2019-04-03') \
        .select(S2bands).mosaic()
# print(s2mosaic.getInfo())

{'type': 'Image', 'bands': [{'id': 'sargassum', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [14814, 39867], 'crs': 'EPSG:32616', 'crs_transform': [10, 0, 394190, 0, -10, 2399960]}], 'version': 1615835007722328, 'id': 'projects/ee-abnatcap/assets/sargassum/classclipped_20190402', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-86.59071049015424, 21.70250552185316], [-86.59078515218833, 21.70259572676351], [-88.02295445876494, 21.699944926321034], [-88.01981477862928, 21.249683233311362], [-88.01675707980556, 20.79948702394507], [-88.01377973662241, 20.349266810702474], [-88.01088164522679, 19.899023086688608], [-88.00806185039394, 19.44875610132152], [-88.00531924686656, 18.99846648213763], [-88.00265295275857, 18.54815459491377], [-88.000061936187, 18.097820806471727], [-87.99998821648875, 18.097731252598233], [-86.59996610706003, 18.099908939559636], [-86.5998712961599, 18.09997837655819], [-8

## Visualization

In [3]:
# Visualization parameters
sargassum_vis = {
    'min': 1, 
    'max': 1, 
    'palette':'fb04ff' # pink
}

# Sentinel-2 bands
BLUE = 'B2' # 490 nmn, 10m
GREEN = 'B3' # 560 nm, 10m
RED = 'B4' # 665 nm, 10m
REDEDGE1 = 'B5' # 705 nm, 20m
REDEDGE2 = 'B6' # 740 nm, 20m
REDEDGE3 = 'B7' # 783 nm, 20m
NIR = 'B8' # 842 nm, 10m
NIR2 = 'B8A' # 865 nm, 20m
SWIR1 = 'B11' # 1610nm, 20m
SWIR2 = 'B12' # 2190 nm, 20m

rgbVis = {
  'min': 0,
  'max': 2500,
  'bands': [RED, GREEN, BLUE],
}
cirVis = {
  'min': 0,
  'max': 2500,
  'bands': [NIR, RED, GREEN],
}

In [4]:
Map = geemap.Map(center=[20,-87], zoom=8)  # Center on Quintana Roo
Map.add_basemap('Esri Ocean')
Map.add_basemap('Esri Satellite')
Map.add_basemap('HYBRID')

In [5]:
# Display Sentinel-2 CIR and natural color mosaics
Map.addLayer(s2mosaic, cirVis, "S2 CIR mosaic - 2019-04-02", False)
Map.addLayer(s2mosaic, rgbVis, "S2 RGB mosaic - 2019-04-02", False)

In [6]:
# Display outline of Nearshore Mask
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();
# Paint all the polygon edges with the same number and width, display.
nearshoreoutline = empty.paint(**{
  'featureCollection': nearshore_mask,
  'color': 1,
  'width': 1.5
});
Map.addLayer(nearshoreoutline,  {'palette':'cyan'}, 'Nearshore mask', True); # orange

In [7]:
# Display classified mosaic
# Classified region, 1=present, 0=absent, -1=no data
p_a_nd_palette = ["white", "blue", "fb04ff"]
Map.addLayer(classified, {'min':-1, 'max':1, 'palette':p_a_nd_palette }, 'Classified', False)
# Sargassum only
Map.addLayer(sargassum, {'min': 1, 'max': 1, 'palette':'42f545'}, 'Sargassum', True)  # cyan

Map

Map(center=[20, -87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## Create Accuracy Assessment Random Samples

In [8]:
## Accuracy Assessment Random Samples
image_prj = classified.select('sargassum').projection()
print('Creating AA points..')
aaPoints = classified.stratifiedSample(
    numPoints=100,
    classBand='sargassum',
    projection=image_prj,
    scale=10,
    region=nearshore_mask.geometry(), 
    dropNulls=False,
    geometries=True
)
print('AA points', aaPoints.size().getInfo())
print('AA Points', aaPoints.first().getInfo())


Creating AA points..
AA points 300
AA Points {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-87.02582294507685, 20.361220501449772]}, 'id': '0', 'properties': {'sargassum': -1}}


In [9]:
# Training points and classified data
vis_params = {
    'color': '000000', 
    'colorOpacity': 1,
    'pointSize': 5,
    'pointShape': 'circle',
    'width': 0.5,
    'lineType': 'solid', 
    'fillColorOpacity': 0.66    
}
Map.add_styled_vector(aaPoints, column="sargassum", palette=p_a_nd_palette, layer_name="land cover", **vis_params)

In [14]:
classed_tile = os.path.join('/Users/arbailey/Google Drive/My Drive/geeout/s2sr_classified', 
                          '20190402T160901_20190402T162422_T16QDH.tif')
Map.add_raster(classed_tile, colormap='terrain', layer_name='clipped tile T16QDH')